In [1]:
from json import loads
from pyspark.sql import SparkSession
import warnings
warnings.filterwarnings("ignore")
from pyspark.sql.functions import col,from_json,udf
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,MapType,FloatType,ArrayType

import os 
# es = "org.elasticsearch:elasticsearch-spark-30_2.12:7.12.1"
# http_scala='org.apache.httpcomponents:httpclient:4.5.13'
# common_client='commons-httpclient:commons-httpclient:3.1'

# os.environ["PYSPARK_SUBMIT_ARGS"] = ("--packages {},{},{} pyspark-shell".format(es, http_scala,common_client))

In [2]:
spark = SparkSession.\
        builder.\
        appName("process-data").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1024m").\
        getOrCreate()

22/11/12 07:22:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/11/12 07:22:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# spark.stop()

In [4]:
data = spark.read.parquet('hdfs://namenode:9000/tiki/Product')

22/11/12 07:22:30 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources


In [5]:
sample_data = data.select('value').take(100)

In [6]:
data_sample = [loads(item.value) for item in sample_data]

In [5]:
schema = StructType([ 
    StructField("id",IntegerType(),True), 
    StructField("master_id",IntegerType(),True), 
    StructField("sku",StringType(),True), 
    StructField("name",StringType(),True),
    StructField("short_description",StringType(),True), 
    StructField("price",IntegerType(),True),
    StructField("list_price",IntegerType(),True),
    StructField('original_price', IntegerType(),True),
    StructField('discount', IntegerType(),True),
    StructField('discount_rate', FloatType(),True),
    
    StructField("rating_average",FloatType(),True), 
    StructField("review_count",IntegerType(),True), 
    StructField("productset_group_name",StringType(),True), 
    StructField("all_time_quantity_sold",IntegerType(),True),
    
    StructField("description",StringType(),True), 
    StructField("current_seller",MapType(StringType(),StringType()),True),
    StructField("breadcrumbs",ArrayType(MapType(StringType(),StringType())),True),
    StructField("specifications",ArrayType(StructType(
                    [
                        StructField("name", StringType()),
                        StructField("attributes",ArrayType(MapType(StringType(),StringType())),True),
                    ]
                )),True),

    StructField('return_and_exchange_policy', StringType(),True)
])

In [6]:
df = data.withColumn("jsonData",from_json(col("value"),schema)) \
                   .select("jsonData.*")

In [7]:
df.createOrReplaceTempView('Product')

In [8]:
def parserAtt(specifications):
    result = ""
    try:
        for s in specifications:
            for a in s.attributes:
                result += a['value']
        result = cleanText(result)
    except:
        return ""
    return result

In [9]:
spark.udf.register('parserAtt',parserAtt,StringType())

<function __main__.parserAtt(specifications)>

In [10]:
# spark.sql("""
#         select current_seller.id seller_id,current_seller.name seller_name,current_seller.store_id seller_store_id,
#         breadcrumbs[0].name category_name,breadcrumbs[0].category_id category_id
#         from Product where specifications[0].name is not null limit 10
# """).collect()

In [13]:
import re
# as per recommendation from @freylis, compile once only
CLEANR = re.compile('<.*?>') 
def cleanText(str_raw):
    # remove tags html
    str_raw = re.sub(CLEANR, ' ', str_raw)

    # remove special character
    str_raw = re.sub('\W+', ' ', str_raw)
    
    # remove number
    str_raw = re.sub("[0-9]+", "", str_raw)
    
    # remove space
    cleantext = re.sub(" +", " ", str_raw)
    return cleantext.lower()

In [15]:
spark.udf.register("cleanText", cleanText,StringType())

<function __main__.cleanText(str_raw)>

In [16]:
product_clean = spark.sql("""
        select id,master_id,sku,price,list_price,original_price,discount,discount_rate,
        rating_average,review_count,productset_group_name,all_time_quantity_sold,
        cleanText(name) name,cleanText(description) description,parserAtt(specifications) specifications,
        current_seller.id seller_id,current_seller.name seller_name,current_seller.store_id seller_store_id,
        cast(current_seller.product_id as int) product_id,
        breadcrumbs[0].name category_name,breadcrumbs[0].category_id category_id
        from Product
""")

In [17]:
product_clean.write.partitionBy("category_id").mode('append').parquet('hdfs://namenode:9000/TikiCleaned/Product')

In [239]:
test = spark.read.parquet('hdfs://namenode:9000/TikiCleaned/Product')

In [240]:
test.first()

Row(id=129734261, master_id=129734261, sku='5403899666753', price=15000, list_price=15000, original_price=15000, discount=0, discount_rate=0.0, rating_average=3.700000047683716, review_count=7, productset_group_name='Voucher - Dịch vụ/Khóa học/Khóa học ngoại ngữ', all_time_quantity_sold=43, name=' e book achievers grade sách mềm sách bài tập', description=' hạn sử dụng đây là sản phẩm nằm trong kho ứng dụng tài nguyên số eduhome sử dụng trên đa nền tảng trên trình duyệt phần mềm trên máy tính window nền tảng mobile trên android và ios thông tin cơ bản của sản phẩm được thể hiện ở hình bên dưới giá bán đã bao gồm vat nhà bán hàng chịu trách nhiệm xuất hóa đơn cho khách hàng', specifications=' khách hàng sau khi nhận mã evoucher vui lòng thực hiện theo hướng dẫn sau thông tin liên hệ về việc hỗ trợ và giải đáp thắc mắc liên quan tới sản phẩm eduhome hotline zalo dtp software customer service zalo me email info dtp education com giá bán đã bao gồm vat nhà bán hàng chịu trách nhiệm xuất hó

In [49]:
import re
# as per recommendation from @freylis, compile once only
CLEANR = re.compile('<.*?>') 

In [65]:
import gensim

In [218]:
def cleanText(str_raw):
    # remove tags html
    str_raw = re.sub(CLEANR, ' ', str_raw)

    # remove special character
    str_raw = re.sub('\W+', ' ', str_raw)
    
    # remove number
    str_raw = re.sub("[0-9]+", "", str_raw)
    
    # remove space
    cleantext = re.sub(" +", " ", str_raw)
    return cleantext.lower()

In [219]:
spark.udf.register("cleanText", cleanText,StringType())

22/11/09 16:22:28 WARN SimpleFunctionRegistry: The function cleantext replaced a previously registered function.


<function __main__.cleanText(str_raw)>

In [220]:
spark.sql("""
        select cleanText(short_description),short_description,description from Product limit 1
""").collect()

[Row(cleanText(short_description)='đây là sản phẩm nằm trong kho ứng dụng tài nguyên số eduhome sử dụng trên đa nền tảng trên trình duyệt phần mềm trên máy tính window nền tảng mobile trên android và iosthông tin cơ bản của sản p ', short_description='Đây là sản phẩm nằm trong kho ứng dụng tài nguyên số Eduhome, sử dụng trên đa nền tảng: trên trình duyệt, Phần mềm trên máy tính window, nền tảng mobile trên Android và iOSThông tin cơ bản của sản p...', description='<p>Hạn sử dụng: 2026-12-31 00:00:00</p><ul>\n<li>Đây là sản phẩm nằm trong kho ứng dụng tài nguyên số Eduhome, sử dụng trên đa nền tảng: trên trình duyệt, Phần mềm trên máy tính window, nền tảng mobile trên Android và iOS</li>\n<li>Thông tin cơ bản của sản phẩm được thể hiện ở hình bên dưới</li>\n</ul>\n<p><img src="https://salt.tikicdn.com/ts/tmp/1a/f6/8e/39b1636be67cee50fb30c128d0421e6a.png" alt="" width="750" height="368" /></p>Giá bán đã bao gồm VAT - Nhà Bán Hàng chịu trách nhiệm xuất hóa đơn cho Khách hàng')]

In [115]:
spark.sql("""
        select cleanText(description) description,cleanText(description) description from Product limit 1
""").collect()

[Row(cleanText(description)='hạn sử dụng đây là sản phẩm nằm trong kho ứng dụng tài nguyên số eduhome sử dụng trên đa nền tảng trên trình duyệt phần mềm trên máy tính window nền tảng mobile trên android và ios thông tin cơ bản của sản phẩm được thể hiện ở hình bên dưới giá bán đã bao gồm vat nhà bán hàng chịu trách nhiệm xuất hóa đơn cho khách hàng', description='<p>Hạn sử dụng: 2026-12-31 00:00:00</p><ul>\n<li>Đây là sản phẩm nằm trong kho ứng dụng tài nguyên số Eduhome, sử dụng trên đa nền tảng: trên trình duyệt, Phần mềm trên máy tính window, nền tảng mobile trên Android và iOS</li>\n<li>Thông tin cơ bản của sản phẩm được thể hiện ở hình bên dưới</li>\n</ul>\n<p><img src="https://salt.tikicdn.com/ts/tmp/1a/f6/8e/39b1636be67cee50fb30c128d0421e6a.png" alt="" width="750" height="368" /></p>Giá bán đã bao gồm VAT - Nhà Bán Hàng chịu trách nhiệm xuất hóa đơn cho Khách hàng')]

In [10]:
shop_id = spark.sql("""select distinct current_seller.id from Product""").collect()

In [11]:
len(shop_id)

6839

In [16]:
x = [int(item.id) for item in shop_id if item.id != None]

In [17]:
len(x)

6838

In [82]:
data_sample[4]['original_price']

97000

In [88]:
for item in data_sample:
    print(item['seller'])

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [86]:
data_sample[6]['seller']

In [81]:
data_sample[4]['discount']

9000

In [70]:
data_sample[a]

{'id': 186116351,
 'master_id': 186116351,
 'sku': '1297824668810',
 'name': 'Giftpop - Phiếu Quà Tặng GoGi House 1000K',
 'url_key': 'giftpop-phieu-qua-tang-gogi-house-1000k-p186116351',
 'url_path': 'giftpop-phieu-qua-tang-gogi-house-1000k-p186116351.html?spid=186116353',
 'short_url': 'https://tiki.vn/product-p186116351.html?spid=186116353',
 'type': 'simple',
 'book_cover': None,
 'short_description': 'Phiếu quà tặng GoGi House với nhiều mệnh giá từ 100k đến 1000k sẽ là món quà ý nghĩa cho người thân, bạn bè, đối tác mỗi dịp Lễ, Tết hoặc các dịp đặc biệt, liên hoan, tiệc công ty, sum họp gia đình.Ph...',
 'price': 1000000,
 'list_price': 1000000,
 'original_price': 1000000,
 'badges': [{'code': 'new_pdp', 'text': 'v1'}],
 'badges_new': [],
 'discount': 0,
 'discount_rate': 0,
 'rating_average': 5,
 'review_count': 2,
 'review_text': '(2)',
 'favourite_count': 0,
 'thumbnail_url': 'https://salt.tikicdn.com/cache/280x280/ts/product/8f/5b/a2/5ef3a12af63683bb0151c0a19b7ba340.jpg',
 'ha